# Operator Definitions
We will define the fermionic creation and annihilation operators $c_j^{\dagger}$ and $c_j$ respectively.

In [6]:
import numpy as np
from dataclasses import dataclass, field
from typing import List, Callable
from numpy.typing import NDArray

In [7]:
Z = np.array([[1, 0], [0, -1]])
creation_operator = np.array([[0, 0], [1, 0]])
annihilation_operator = np.array([[0, 1], [0, 0]])

In [8]:
# Antisymmetry Confirmations for Fermionic Operators 
vacuum_state = np.array([1, 0])
occupied_state = np.array([0, 1])

# c† |0> = |1>
assert((creation_operator @ vacuum_state == occupied_state).all())

# c† |0> = NULL
assert((creation_operator @ occupied_state == np.zeros(2,)).all())

# c |1> = |0>
assert((annihilation_operator @ occupied_state == vacuum_state).all())

# c† |0> = NULL
assert((annihilation_operator @ vacuum_state == np.zeros(2,)).all())

# c†c + cc† = I
assert((annihilation_operator @ creation_operator + creation_operator @ annihilation_operator == np.eye(2)).all())

In [9]:
def creation_gate(i, N):
    ops = np.eye(2**N)

    for j in range(i):
        Z_j = np.kron(np.eye(2**j), np.kron(Z, np.eye(2**(N-j-1))))
        ops = ops @ Z_j
    
    c_i = np.kron(np.eye(2**i), np.kron(creation_operator, np.eye(2**(N-i-1))))
    ops = ops @ c_i
    
    return ops

def annihilation_gate(i, N): 
    ops = np.eye(2**N)

    for j in range(i):
        Z_j = np.kron(np.eye(2**j), np.kron(Z, np.eye(2**(N-j-1))))
        ops = ops @ Z_j
    
    c_i = np.kron(np.eye(2**i), np.kron(annihilation_operator, np.eye(2**(N-i-1))))
    ops = ops @ c_i
    
    return ops

In [10]:
# Required Classes

@dataclass 
class KitaevChain: 
    N: int
    mu: Callable[[float], float]
    Delta: Callable[[float], float]
    h: Callable[[float], float]
    t0: float
    creations: NDArray = field(init=False)
    annihilations: NDArray = field(init=False)
    hamiltonian: NDArray = field(init=False)
    chemical_potential_term: NDArray = field(init=False)
    hopping_term: NDArray = field(init=False)
    pairing_term: NDArray = field(init=False)
    state_vector : NDArray = field(init=False)

    def __post_init__(self):
        # Create and store creation and annihilation operators for each fermionic mode
        self.creations = np.zeros((self.N, 2**self.N, 2**self.N))
        self.annihilations = np.zeros((self.N, 2**self.N, 2**self.N))
        
        for i in range(self.N): 
            self.creations[i] = (creation_gate(i, self.N))
            self.annihilations[i] = (annihilation_gate(i, self.N))

        # Favors or penalizes particle occupations depending on sign and magnitude of mu
        self.chemical_potential_term = np.einsum('ijk,ilk->jk', self.creations, self.annihilations)

        # Delocalizes fermion wavefunctions, allows them to spread apart on the chain
        self.hopping_term = (
            np.einsum('ijk, ilk -> jk', self.creations[:self.N-1], self.annihilations[1:self.N])
            + np.einsum('ijk, ilk -> jk', self.creations[1:self.N], self.annihilations[:self.N-1])
        )
        
        # p-wave superconducting correlations, where electrons pair across neighboring sites
        self.pairing_term = ( 
            np.einsum('ijk, ilk -> jk', self.annihilations[:self.N-1], self.annihilations[1:self.N])
            + np.einsum('ijk, ilk -> jk', self.creations[1:self.N], self.creations[:self.N-1])
        )

        self.hamiltonian = (
            -self.mu(self.t0) * self.chemical_potential_term 
            - self.h(self.t0) * self.hopping_term 
            + self.Delta(self.t0) * self.pairing_term
            )

        self.state_vector = np.zeros(2**self.N,)
        self.state_vector[0] = 1

    def evolve(self, t): 
        pass

    def __repr__(self): 
        return f'{self.__class__.__name__} with {self.N} fermionic modes'